In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
activity = pd.read_csv("user_vector.csv").drop("Unnamed: 0", axis=1)
label = pd.read_csv("data/train_label.csv").drop("Unnamed: 0", axis=1)

In [4]:
activity.head()

,acc_id,week,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,duel_win,partybattle_cnt,partybattle_win,cnt_enter_inzone_solo,cnt_enter_inzone_light,cnt_enter_inzone_skilled,cnt_enter_inzone_normal,cnt_enter_raid,cnt_enter_raid_light,cnt_enter_bam,cnt_clear_inzone_solo,cnt_clear_inzone_light,cnt_clear_inzone_skilled,cnt_clear_inzone_normal,cnt_clear_raid,cnt_clear_raid_light,cnt_clear_bam,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,party_count,party_time_total,party_time_mean,trade_count,trade_accessory_cnt,trade_costume_cnt,trade_gem_cnt,trade_grocery_cnt,trade_money_cnt,trade_weapon_cnt,trade_accessory_total,trade_costume_total,trade_gem_total,trade_grocery_total,trade_money_total,trade_weapon_total
0,18,1,1.234966,1.118680,-0.231874,0.620894,-0.250423,0.775216,0.549571,0.905233,-0.014795,0.137556,0.208674,6.475093,6.292342,-0.308749,0.593175,-0.190671,0.625977,1.124363,1.728465,-0.047643,-0.306502,0.586938,-0.188788,0.635345,0.997163,1.778959,-0.03313,0.181697,2.577756,0.012339,2.585082,2.413561,-0.024028,2.410632,-0.027758,3.504878,0.025883,1.439230,1.091208,0.072041,-0.020544,-0.034362,-0.040215,-0.032042,-0.028296,-0.012704,-0.015583,0.0,0.0,0.0,0.000000,0.914091,0.0
1,18,2,1.234966,2.027186,-0.231874,1.446126,-0.250423,2.092845,0.333512,1.805012,0.013952,0.028978,0.058674,3.996524,4.204861,-0.030266,1.769260,-0.190671,1.710292,2.616856,3.148809,-0.047643,-0.026092,1.708467,-0.188788,1.631109,1.679553,3.232370,-0.03313,0.043768,2.195751,-0.016574,2.619647,1.824373,-0.024028,2.908992,-0.027758,3.142027,-0.149898,2.439523,5.180487,0.542469,-0.008389,-0.034362,-0.040215,-0.032042,-0.028296,0.011631,-0.015583,0.0,0.0,0.0,0.000000,10.212594,0.0
2,18,3,1.234966,0.992366,-0.231874,0.574930,-0.250423,1.057040,0.154848,0.983988,0.032468,0.072409,0.108674,5.979380,6.102571,-0.308749,0.457472,0.408652,0.448946,2.119358,3.858982,-0.047643,-0.306502,0.352199,0.424934,0.354489,2.361942,3.959076,-0.03313,0.404505,1.148319,0.002702,2.459784,1.300954,-0.024028,2.123533,-0.027758,0.964919,0.553227,1.465554,0.139402,-0.136277,-0.012441,-0.034362,-0.040215,-0.032042,-0.024254,-0.000537,-0.015583,0.0,0.0,0.0,-0.056346,0.480802,0.0
3,18,4,1.234966,2.086213,-0.231874,1.277514,-0.250423,1.703230,1.816838,2.180256,0.055894,0.745590,1.008672,8.061378,8.379823,0.248218,0.638409,2.506282,0.980039,2.616856,3.148809,5.512386,0.254319,0.743430,2.572960,1.094929,3.044332,3.232370,-0.03313,0.468165,1.357805,-0.016574,3.950401,3.315157,-0.024028,3.434437,-0.027758,2.658225,-0.149898,1.834082,2.282504,0.230806,0.003765,-0.034362,-0.040215,-0.032042,-0.020212,0.027854,-0.015583,0.0,0.0,0.0,-0.112692,4.900587,0.0
4,18,5,1.234966,1.464852,-0.231874,0.874912,-0.250423,1.943185,3.998198,1.285647,-0.013040,0.224418,0.308673,7.565664,7.051426,-0.123093,0.593175,0.708313,0.825137,3.114353,3.858982,-0.047643,-0.119562,0.639102,0.731795,0.916202,3.726722,3.959076,-0.03313,0.531824,0.569151,-0.016574,2.520273,2.029903,-0.024028,2.356462,-0.027758,1.448721,-0.114742,1.834082,0.142991,-0.153807,-0.022569,-0.034362,-0.040215,-0.032042,-0.028296,-0.016760,-0.015583,0.0,0.0,0.0,0.000000,0.892256,0.0


In [5]:
label = label.sort_values("acc_id")
label.head()

,acc_id,label
75988,18,retained
870,20,week
4600,21,week
50725,25,2month
87284,26,retained


<br></br><br></br><br></br>

In [6]:
id_lst = [int(x) for x in label.acc_id.tolist()]
print(len(id_lst))

100000


In [7]:
activity_dic = {}
for user in id_lst :
    activity_dic[user] = []

In [8]:
print(len(activity.head().values[0]))
activity.head().values[0]

55


array([  1.80000000e+01,   1.00000000e+00,   1.23496602e+00,
         1.11867967e+00,  -2.31874044e-01,   6.20893833e-01,
        -2.50423355e-01,   7.75215810e-01,   5.49570822e-01,
         9.05232805e-01,  -1.47949164e-02,   1.37556005e-01,
         2.08673646e-01,   6.47509339e+00,   6.29234215e+00,
        -3.08749015e-01,   5.93174522e-01,  -1.90671430e-01,
         6.25976613e-01,   1.12436303e+00,   1.72846454e+00,
        -4.76429006e-02,  -3.06502306e-01,   5.86937722e-01,
        -1.88787866e-01,   6.35345248e-01,   9.97163150e-01,
         1.77895908e+00,  -3.31304237e-02,   1.81697236e-01,
         2.57775553e+00,   1.23393065e-02,   2.58508246e+00,
         2.41356116e+00,  -2.40280218e-02,   2.41063182e+00,
        -2.77583111e-02,   3.50487846e+00,   2.58828290e-02,
         1.43923023e+00,   1.09120772e+00,   7.20411949e-02,
        -2.05435186e-02,  -3.43617618e-02,  -4.02151741e-02,
        -3.20419810e-02,  -2.82960163e-02,  -1.27042052e-02,
        -1.55832646e-02,

In [9]:
for data in activity.values :
    activity_dic[int(data[0])].append([int(data[1])]+list(data[2:]))

In [10]:
activity_lst = [list(y) for y in activity_dic.items()]
activity_lst[0]

[18,
 [[1,
   1.2349660199753358,
   1.1186796652143001,
   -0.23187404383711799,
   0.62089383262644704,
   -0.25042335545275801,
   0.775215810330808,
   0.549570821725343,
   0.90523280478207724,
   -0.014794916430443599,
   0.13755600538371099,
   0.208673645558455,
   6.4750933917829201,
   6.2923421457450903,
   -0.30874901478870698,
   0.59317452243608604,
   -0.19067142960053896,
   0.62597661318211595,
   1.12436302721398,
   1.7284645374228702,
   -0.047642900560293501,
   -0.306502306156819,
   0.5869377215598921,
   -0.18878786561800398,
   0.63534524751305699,
   0.99716315040983405,
   1.7789590810198999,
   -0.033130423698603299,
   0.18169723577544,
   2.5777555289285097,
   0.0123393064586017,
   2.5850824646089903,
   2.4135611557066001,
   -0.024028021828956601,
   2.41063181722923,
   -0.027758311120298699,
   3.5048784613889299,
   0.025882829045283402,
   1.4392302312375618,
   1.0912077245089902,
   0.072041194921310406,
   -0.020543518568428055,
   -0.0343617617

In [11]:
day_1_lst = [x for x in activity_lst]
print(len(day_1_lst))

100000


In [12]:
day_1_dic = {}
day_1_id_lst = [x[0] for x in day_1_lst]

In [13]:
START_TOKEN = [1] + [0]*52
EMPTY_TOKEN = [0,1] + [0]*51
END_TOKEN = [0,0,1] + [0]*50

In [14]:
for user in day_1_id_lst :
    day_1_dic[(user, 0)] = START_TOKEN
    day_1_dic[(user, 1)] = EMPTY_TOKEN
    day_1_dic[(user, 2)] = EMPTY_TOKEN
    day_1_dic[(user, 3)] = EMPTY_TOKEN
    day_1_dic[(user, 4)] = EMPTY_TOKEN
    day_1_dic[(user, 5)] = EMPTY_TOKEN
    day_1_dic[(user, 6)] = EMPTY_TOKEN
    day_1_dic[(user, 7)] = EMPTY_TOKEN
    day_1_dic[(user, 8)] = EMPTY_TOKEN
    day_1_dic[(user, 9)] = END_TOKEN


In [15]:
for data in day_1_lst :
    user = data[0]
    lst = data[1]
    
    for idx, data2 in enumerate(lst) :
        if idx == 0 :
            first_week = data2[0]
            
            for idx2 in range(first_week) :
                day_1_dic[(user, idx2)] = START_TOKEN
                
        week = data2[0]
        day_1_dic[(user, week)] = data2[1:]

In [16]:
label_dic = {"week":0 , "month" :1, "2month":2, "retained":3}

In [17]:
label2 = label.sort_values(by="acc_id")
label2["label"] = label2["label"].map(lambda x : label_dic[x])

label2.head()

,acc_id,label
75988,18,3
870,20,0
4600,21,0
50725,25,2
87284,26,3


<br></br><br></br><br></br>

In [18]:
def one_hot(lst, num_class=4) :
    return np.eye(num_class)[lst]

In [19]:
day_1_total_lst = []
temp1 = list(day_1_dic.values())

last1=0

In [20]:
for now in range(0,len(temp1)+1,10) :
    if now == 0 :
        last1 = now
        continue
    
    day_1_total_lst.append(temp1[last1:now])
    last1=now

In [21]:
print(len(day_1_total_lst))

100000


In [22]:
label_dic = label2.set_index("acc_id").to_dict()['label']
print(len(label_dic))

100000


In [23]:
day_1_total_label = one_hot([label_dic[x] for x in day_1_id_lst])

In [24]:
print(len(day_1_total_label))

100000


In [25]:
idx1 = len(day_1_total_lst)//5 *4

day_1_training_lst = np.array(day_1_total_lst[:idx1])
day_1_valid_lst = np.array(day_1_total_lst[idx1:])

day_1_training_label = np.array(day_1_total_label[:idx1])
day_1_valid_label = np.array(day_1_total_label[idx1:])

In [26]:
print(np.array(day_1_total_lst).shape)
print(np.array(day_1_total_label).shape)

(100000, 10, 53)
(100000, 4)


<br></br><br></br><br></br>

# Model
- Transformer with denseNet
    - Encoding된 vector를 stack하는 방식
    - 마지막에 1x1 CNN
    
    
- need to customize
    - QK_T_dk -> QK_T
    - sinusoid PE -> other PE
    - pooling
    - one-hot을 한 후 cost 계산
    
    
- https://github.com/jadore801120/attention-is-all-you-need-pytorch/blob/4f4a192f0fd272102c8852b00b1007dffd292b90/transformer/Models.py#L11
- https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
- http://nlp.seas.harvard.edu/2018/04/03/attention.html


In [27]:
class Transformer_Classifer() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
    def enc_embedding(self, X_input, emb_dim, emb_activation) :
        if emb_dim == X_input.get_shape().as_list()[2] :
            emb_vector = X_input
            
        else :
            batch, position, dim = X_input.get_shape().as_list()
            W_emb = tf.Variable(tf.random_normal([batch, dim, emb_dim]), name="W_emb_enc")
            emb_vector = tf.matmul(X_input, W_emb)
            
        return emb_vector
    
    def dec_embedding(self, X_input, emb_dim, emb_activation) :
        batch, position, dim = X_input.get_shape().as_list()
        dimension = position*dim
        
        flat = tf.reshape(X_input, [batch, 1, dimension])
        W_emb = tf.Variable(tf.random_normal([batch, dimension, emb_dim]), name="W_emb_dec")
        
        emb_vector = tf.matmul(flat, W_emb)
        return emb_vector
    
    def positional_encoding(self, X_input) :
        batch, position, dim = X_input.get_shape().as_list()
        position_enc = np.array([list([[pos / np.power(10000, 2*i/dim) for i in range(dim)] 
                                if pos != 0 else np.zeros(dim) for pos in range(position)])]*batch)

        position_enc[:, 1:, 0::2] = np.sin(position_enc[:, 1:, 0::2])
        position_enc[:, 1:, 1::2] = np.cos(position_enc[:, 1:, 1::2])

        pos_enc_vector = tf.constant(position_enc, dtype=tf.float32, shape=[batch, position, dim])
        return pos_enc_vector
    
    def scaled_dot_product_attention(self, Q, K, V, dk) :
        QKT_dk = tf.matmul(Q,K, transpose_b=True) / dk
        attention = tf.nn.softmax(QKT_dk)
        attended_vector = tf.matmul(attention,V)
        
        return attended_vector
        
    def multihead_attention(self, Q, K, V, h) :
        batch, position, dim_q = Q.get_shape().as_list()
        batch, position, dim_k = K.get_shape().as_list()
        batch, position, dim_v = V.get_shape().as_list()
        dq = dim_q//h
        dk = dim_k//h
        dv = dim_v//h
        
        W_Q = tf.Variable(tf.random_normal([h, batch, dim_q, dq]), name="W_Q")
        W_K = tf.Variable(tf.random_normal([h, batch, dim_k, dk]), name="W_K")
        W_V = tf.Variable(tf.random_normal([h, batch, dim_v, dv]), name="W_V")
        W_O = tf.Variable(tf.random_normal([batch, dk*h, dim_k]), name="W_O")
    
        head_lst = []
        for idx in range(h) :
            head_lst.append(self.scaled_dot_product_attention(tf.matmul(Q, W_Q[idx]), 
                                                              tf.matmul(K, W_K[idx]), 
                                                              tf.matmul(V, W_V[idx]), 
                                                              tf.constant(dk, tf.float32)))
        
        multihead_attention = tf.concat(head_lst, axis=2)
        linear_projection= tf.matmul(multihead_attention, W_O)
        
        return linear_projection
        
    def normarlization(self, X_input) :
        return tf.contrib.layers.layer_norm(X_input)
    
    def feedforward_network(self, X_input, ffn_dim) :
        batch, position, dim =  X_input.get_shape().as_list()

        ff_vector1 = tf.layers.dense(X_input, ffn_dim, activation=tf.nn.relu)
        dropout = tf.layers.dropout(ff_vector1, training=self.training)
        ff_vector2 = tf.layers.dense(dropout, dim)
        
        return ff_vector2
    
    def resnet(self, X_input1, X_input2) :
        return X_input1 + X_input2
        
    def encoder_layer(self, X_input, h, ffn_dim) :
        attended_vector = self.multihead_attention(X_input, X_input, X_input, h)
        sublayer1 = self.resnet(X_input, attended_vector)
        norm1 = self.normarlization(sublayer1)
        
        ffn_vector = self.feedforward_network(norm1, ffn_dim)
        sublayer2 = self.resnet(norm1, ffn_vector)
        norm2 = self.normarlization(sublayer2)
        
        return norm2
    
    def decoder_layer(self, X_input, Y_input, h, ffn_dim) :
        attended_vector = self.multihead_attention(Y_input, X_input, X_input, h)
        sublayer1 = self.resnet(X_input, attended_vector)
        norm1 = self.normarlization(sublayer1)
        
        ffn_vector = self.feedforward_network(norm1, ffn_dim)
        sublayer2 = self.resnet(norm1, ffn_vector)
        norm2 = self.normarlization(sublayer2)
        
        return norm2
    
    def Encode(self) :
        enc_embedded_vector = self.enc_embedding(self.X, self.emb_dim, self.emb_activation)
        enc_pos_encoded_vector = enc_embedded_vector + self.positional_encoding(enc_embedded_vector)
        encoder_input = enc_pos_encoded_vector
        
        for idx in range(self.N) :
            encoder_input = self.encoder_layer(encoder_input, self.h, self.ffn_dim)
        encoder_output = encoder_input
    
        return encoder_output
    
    def Decode(self, encoder_output) :
        dec_embedded_vector = self.dec_embedding(self.X, self.emb_dim, self.emb_activation)
        decoder_input = dec_embedded_vector
        
        W_D = tf.Variable(tf.random_normal([self.output_length-1, self.batch_size, self.emb_dim, self.emb_dim]), name="W_D")
        decode_lst = []

        for idx in range(self.output_length) :
            if idx != 0 :
                decoder_input = tf.matmul(decoder_input, W_D[idx-1])
                
            for idx2 in range(self.N) :
                decoder_input = self.decoder_layer(encoder_output, decoder_input, self.h, self.ffn_dim)
                
            decoder_output = decoder_input
            decode_lst.append(decoder_output)

        return decode_lst
    
    def Classify(self, decoder_output) : 
        classifier_input = tf.stack(decoder_output, axis=1)
        batch, length, position, dim = classifier_input.get_shape().as_list()

        dimension = position*dim
        flat = tf.reshape(classifier_input, [batch, length, dimension])
        layer = tf.layers.dense(inputs=flat, units=1)

        return tf.reshape(layer, [batch, length])
    
    def build(self, batch_size, input_length, output_length, input_dim, N, emb_dim, emb_activation, h, ffn_dim, fc_activation) :
        with tf.variable_scope(self.name) :
            
            ## Setting ##
            # input  : ? x input_length x input_dim
            self.X = tf.placeholder(tf.float32, [batch_size, input_length, input_dim])
            self.Y = tf.placeholder(tf.float32, [batch_size, output_length])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            
            self.batch_size = batch_size
            self.input_length = input_length
            self.output_length = output_length
            self.input_dim = input_dim
            self.N = N
            self.emb_dim = emb_dim
            self.emb_activation = emb_activation
            self.h = h 
            self.ffn_dim = ffn_dim
            self.fc_activation = fc_activation
            #############
            
            
            ## Encoder and Decoder ##
            self.Encoder = self.Encode()
            self.Decoder = self.Decode(self.Encoder)
            #########################
            
            
            ## Classifier ##
            self.logit = self.Classify(self.Decoder)
            self.softmax = tf.nn.softmax(self.logit)
            self.softmax_logit = tf.nn.softmax_cross_entropy_with_logits(logits=self.logit, labels=self.Y)
            ################
            
            
            ## Learning ##
            self.cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logit, labels=self.Y))

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            
            self.prediction = tf.equal(tf.argmax(self.logit, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(self.prediction, tf.float32))    
            ##############
        
        
    def train(self, X_input, Y_input, learning_rate, training=True):
        feed_dict = {self.X: X_input, self.Y: Y_input, self.learning_rate: learning_rate, self.training: training}
        _, cost = self.sess.run([self.optimizer, self.cost], feed_dict=feed_dict)
        
        return _, cost
    
    def predict(self, X_input, training=False):
        size = X_input.shape[0]
        result_lst = []
        
        total_loss = 0
        total_acc = 0
            
        for idx in range(0, size, self.batch_size):
            X_batch = X_input[idx:idx + batch_size]
            feed_dict = {self.X: X_batch, self.training: False}
                
            result = self.sess.run([self.logit], feed_dict=feed_dict)
            result_lst.append(result)
            
        return np.concatenate([x[0][:] for x in result_lst], axis=0)
    
    def evaluate(self, X_input, Y_input):
        size = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for idx in range(0, size, self.batch_size):
            X_batch = X_input[idx:idx + batch_size]
            Y_batch = Y_input[idx:idx + batch_size]
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= size
        total_acc /= size
            
        return total_loss, total_acc
        
        

In [28]:
tf.reset_default_graph() 

<br></br><br></br><br></br> 

In [29]:
learning_rate1 = 0.05
learning_rate2 = 0.02
learning_rate3 = 0.01
learning_rate4 = 0.005

total_epoch = 80
batch_size = 500

In [30]:
day_1_train_losses1 = []
day_1_train_accs1 = []
day_1_valid_losses1 = []
day_1_valid_accs1 = []

day_1_train_losses2 = []
day_1_train_accs2 = []
day_1_valid_losses2 = []
day_1_valid_accs2 = []

day_1_train_losses3 = []
day_1_train_accs3 = []
day_1_valid_losses3 = []
day_1_valid_accs3 = []

In [ ]:
sess= tf.Session()

day_1_model1 = Transformer_Classifer(sess, "model1")
day_1_model2 = Transformer_Classifer(sess, "model2")
day_1_model3 = Transformer_Classifer(sess, "model3")

day_1_model1.build(batch_size, 10, 4, 53, 6, 64, None, 6, 64, None)
day_1_model2.build(batch_size, 10, 4, 53, 6, 128, None, 6, 128, None)
day_1_model3.build(batch_size, 10, 4, 53, 6, 256, None, 6, 256, None)

sess.run(tf.global_variables_initializer())

In [ ]:
print("Ready!")

In [261]:
print('Learning Started!')
print("")

# train my model
for epoch in range(total_epoch):
    avg_cost1 = 0
    avg_cost2 = 0
    avg_cost3 = 0
    
    total_batch = int(len(day_1_training_lst) / batch_size)
    idx = 0
    
    if epoch == 0 :
        learning_rate = learning_rate1
    elif epoch == 40 :
        learning_rate = learning_rate2
    elif epoch == 70 :
        learning_rate = learning_rate3
    elif epoch == 80 :
        learning_rate = learning_rate4

    for i in range(total_batch):
        batch_xs, batch_ys = day_1_training_lst[idx:idx+batch_size],day_1_training_label[idx:idx+batch_size]
        
        _, c1 = day_1_model1.train(batch_xs, batch_ys, learning_rate)
        _, c2 = day_1_model2.train(batch_xs, batch_ys, learning_rate)
        _, c3 = day_1_model3.train(batch_xs, batch_ys, learning_rate)
        
        avg_cost1 += c1 / total_batch
        avg_cost2 += c2 / total_batch
        avg_cost3 += c3 / total_batch
        
        idx += batch_size
        
        if i%10 == 0 :
            print("log :", i)
            
    #train cost & acc
    cost1, acc1 = day_1_model1.evaluate(day_1_training_lst, day_1_training_label)
    cost2, acc2 = day_1_model2.evaluate(day_1_training_lst, day_1_training_label)
    cost3, acc3 = day_1_model3.evaluate(day_1_training_lst, day_1_training_label)
    
    day_1_train_losses1.append(cost1)
    day_1_train_accs1.append(acc1)
    day_1_train_losses2.append(cost2)
    day_1_train_accs2.append(acc2)
    day_1_train_losses3.append(cost3)
    day_1_train_accs3.append(acc3)
    
    #valid cost & acc
    v_cost1, v_acc1 = day_1_model1.evaluate(day_1_valid_lst, day_1_valid_label)
    v_cost2, v_acc2 = day_1_model2.evaluate(day_1_valid_lst, day_1_valid_label)
    v_cost3, v_acc3 = day_1_model3.evaluate(day_1_valid_lst, day_1_valid_label)
    
    day_1_valid_losses1.append(v_cost1)
    day_1_valid_accs1.append(v_acc1)
    day_1_valid_losses2.append(v_cost2)
    day_1_valid_accs2.append(v_acc2)
    day_1_valid_losses3.append(v_cost3)
    day_1_valid_accs3.append(v_acc3)
    
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost1, acc1*100, v_cost1, v_acc1*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost2, acc2*100, v_cost2, v_acc2*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost3, acc3*100, v_cost3, v_acc3*100))
    print('Accuracy:', day_1_model1.evaluate(day_1_valid_lst, day_1_valid_label)[1])
    print('Accuracy:', day_1_model2.evaluate(day_1_valid_lst, day_1_valid_label)[1])
    print('Accuracy:', day_1_model3.evaluate(day_1_valid_lst, day_1_valid_label)[1])
    
    print("train F1 score :", f1_score(np.argmax(day_1_training_label, 1), np.argmax(day_1_model1.predict(day_1_training_lst), 1), average="weighted"))
    print("train F1 score :", f1_score(np.argmax(day_1_training_label, 1), np.argmax(day_1_model2.predict(day_1_training_lst), 1), average="weighted"))
    print("train F1 score :", f1_score(np.argmax(day_1_training_label, 1), np.argmax(day_1_model3.predict(day_1_training_lst), 1), average="weighted"))
    print("valid F1 score :", f1_score(np.argmax(day_1_valid_label, 1), np.argmax(day_1_model1.predict(day_1_valid_lst), 1), average="weighted"))
    print("valid F1 score :", f1_score(np.argmax(day_1_valid_label, 1), np.argmax(day_1_model2.predict(day_1_valid_lst), 1), average="weighted"))
    print("valid F1 score :", f1_score(np.argmax(day_1_valid_label, 1), np.argmax(day_1_model3.predict(day_1_valid_lst), 1), average="weighted"))
    print(" ")

print("")
print('Learning Finished!')

Learning Started!

log : 0
log : 10
log : 20
log : 30
log : 40
log : 50
log : 60
log : 70
log : 80
log : 90
log : 100
log : 110
log : 120
log : 130
log : 140
log : 150


TypeError: evaluate() got an unexpected keyword argument 'batch_size'

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_losses"+str(idx)), label='training'+str(idx))
    plt.plot(eval("day_1_valid_losses"+str(idx)), label='valid'+str(idx))
    plt.title("model"+str(idx))
    plt.grid("on")
    plt.legend()
    plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_accs"+str(idx)), label='training'+str(idx))
    plt.plot(eval("day_1_valid_accs"+str(idx)), label='valid'+str(idx))
    plt.title("model"+str(idx))
    plt.grid("on")
    plt.legend()
    plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_losses"+str(idx)), label='training'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_valid_losses"+str(idx)), label='training'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_accs"+str(idx)), label='valid'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_valid_accs"+str(idx)), label='valid'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
#tf.reset_default_graph() 

<br></br><br></br><br></br>

In [364]:
saver = tf.train.Saver()
saver.save(sess, './model_Transformer/original_user_vector_with_Decoder/original_user_vector_with_Decoder')

'./advanced_RNN/original_user_vector/original'

<br></br><br></br><br></br>

# Test

In [365]:
activity = pd.read_csv("user_vector_test.csv").drop("Unnamed: 0", axis=1)

In [366]:
activity.head()

,acc_id,week,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,duel_win,partybattle_cnt,partybattle_win,cnt_enter_inzone_solo,cnt_enter_inzone_light,cnt_enter_inzone_skilled,cnt_enter_inzone_normal,cnt_enter_raid,cnt_enter_raid_light,cnt_enter_bam,cnt_clear_inzone_solo,cnt_clear_inzone_light,cnt_clear_inzone_skilled,cnt_clear_inzone_normal,cnt_clear_raid,cnt_clear_raid_light,cnt_clear_bam,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,party_count,party_time_total,party_time_mean,trade_count,trade_accessory_cnt,trade_costume_cnt,trade_gem_cnt,trade_grocery_cnt,trade_money_cnt,trade_weapon_cnt,trade_accessory_total,trade_costume_total,trade_gem_total,trade_grocery_total,trade_money_total,trade_weapon_total
0,7,1,-0.684674,-0.458998,-0.231874,-0.116952,-0.250423,-0.085234,0.098756,-0.382474,-0.023541,-0.036168,-0.041326,0.427384,0.788983,-0.308749,-0.424591,-0.190671,-0.458339,2.119358,1.018292,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,3.044332,1.052253,-0.03313,-0.026964,0.199469,-0.016574,-0.098028,0.374695,-0.024028,-0.384519,-0.027758,-0.365536,-0.149898,-0.354297,-0.244665,0.227967,-0.028279,-0.034997,-0.03209,-0.034694,-0.037692,-0.018730,-0.02135,0.0,0.0,0.0,0.0,0.486149,0.0
1,7,2,0.270756,-0.315445,-0.231874,-0.075152,-0.250423,-0.199098,-0.285579,-0.204993,-0.023540,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339,2.616856,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,3.044332,-0.401158,-0.03313,-0.026964,0.026950,-0.016574,-0.028898,0.012960,-0.024028,-0.216593,-0.027758,-0.365536,-0.149898,-0.337451,-0.134934,0.444878,0.012653,-0.034997,-0.03209,-0.034694,-0.037692,0.063212,-0.02135,0.0,0.0,0.0,0.0,6.435135,0.0
2,7,3,-0.206959,-0.499916,-0.231874,-0.058281,-0.250423,-0.228462,-0.306354,-0.416738,-0.023538,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339,2.119358,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,2.361942,-0.401158,-0.03313,-0.023428,-0.182536,-0.016574,-0.102349,-0.167907,-0.024028,-0.319515,-0.027758,-0.365536,-0.149898,-0.438524,-0.275792,0.515011,-0.031690,-0.034997,-0.03209,-0.034694,-0.037692,-0.025559,-0.02135,0.0,0.0,0.0,0.0,0.630145,0.0
3,7,4,-1.162389,-0.554631,-0.231874,-0.102363,-0.250423,-0.258671,-0.306354,-0.482471,-0.023540,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339,2.616856,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,2.361942,-0.401158,-0.03313,-0.034038,-0.231827,-0.016574,-0.253571,-0.244638,-0.024028,-0.379102,-0.027758,-0.365536,-0.149898,-0.472215,-0.355339,0.353108,-0.031690,-0.034997,-0.03209,-0.034694,-0.037692,-0.025559,-0.02135,0.0,0.0,0.0,0.0,0.430349,0.0
4,7,5,-1.162389,-0.565263,-0.231874,-0.156772,-0.250423,-0.258671,0.142383,-0.475183,-0.023540,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339,2.119358,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,2.361942,-0.401158,-0.03313,-0.030501,-0.231827,-0.016574,-0.206044,-0.269302,-0.024028,-0.417021,-0.027758,-0.365536,-0.149898,-0.472215,-0.349680,0.393397,-0.031690,-0.034997,-0.03209,-0.034694,-0.037692,-0.025559,-0.02135,0.0,0.0,0.0,0.0,0.478126,0.0


In [368]:
label = activity.groupby("acc_id").count().reset_index()[["acc_id"]]
label.head()

,acc_id
0,7
1,10
2,15
3,46
4,60


In [369]:
id_lst = [int(x) for x in label.acc_id.tolist()]
print(len(id_lst))

40000


In [370]:
activity_dic = {}
for user in id_lst :
    activity_dic[user] = []

In [371]:
print(len(activity.head().values[0]))
activity.head().values[0]

55


array([ 7.        ,  1.        , -0.68467381, -0.45899802, -0.23187404,
       -0.11695244, -0.25042336, -0.08523406,  0.09875626, -0.38247381,
       -0.02354074, -0.03616804, -0.04132571,  0.42738411,  0.78898263,
       -0.30874901, -0.42459147, -0.19067143, -0.45833892,  2.11935814,
        1.01829211, -0.0476429 , -0.30650231, -0.43026334, -0.18878787,
       -0.4625481 ,  3.04433207,  1.05225341, -0.03313042, -0.02696439,
        0.19946861, -0.01657386, -0.09802801,  0.37469474, -0.02402802,
       -0.38451894, -0.02775831, -0.36553646, -0.14989847, -0.35429675,
       -0.2446647 ,  0.2279671 , -0.02827936, -0.03499703, -0.03209019,
       -0.03469393, -0.03769195, -0.01873001, -0.02135013,  0.        ,
        0.        ,  0.        ,  0.        ,  0.48614946,  0.        ])

In [372]:
for data in activity.values :
    activity_dic[int(data[0])].append([int(data[1])]+list(data[2:]))

In [373]:
activity_lst = [list(y) for y in activity_dic.items()]
activity_lst[0]

[7,
 [[1,
   -0.6846738087407569,
   -0.45899802276336299,
   -0.23187404383711799,
   -0.116952440421407,
   -0.25042335545275801,
   -0.085234061463040897,
   0.098756258865567395,
   -0.38247380803120201,
   -0.023540738832577499,
   -0.036168043070860703,
   -0.041325706121907099,
   0.42738411100963503,
   0.78898263244812783,
   -0.30874901478870698,
   -0.42459146738591708,
   -0.19067142960053896,
   -0.458338915202331,
   2.1193581446815299,
   1.0182921058650001,
   -0.047642900560293501,
   -0.306502306156819,
   -0.43026333989039511,
   -0.18878786561800398,
   -0.46254810329840401,
   3.0443320677336096,
   1.05225341481391,
   -0.033130423698603299,
   -0.026964386020559901,
   0.19946860552136891,
   -0.0165738550449223,
   -0.098028008606172903,
   0.37469474336255998,
   -0.024028021828956601,
   -0.38451893799438902,
   -0.027758311120298699,
   -0.36553646455217803,
   -0.14989847087425101,
   -0.35429674592707633,
   -0.2446647014980104,
   0.22796709506008911,
   -

In [374]:
day_1_lst = [x for x in activity_lst if x[1][0][0]==1]
print(len(day_1_lst))

13502
4880
2073
2108
1514
2294
3275
10354

40000


In [375]:
day_1_dic = {}
day_1_id_lst = [x[0] for x in day_1_lst]

In [376]:
START_TOKEN = [1] + [0]*52
EMPTY_TOKEN = [0,1] + [0]*51
END_TOKEN = [0,0,1] + [0]*50

In [377]:
for user in day_1_id_lst :
    day_1_dic[(user, 0)] = START_TOKEN
    day_1_dic[(user, 1)] = EMPTY_TOKEN
    day_1_dic[(user, 2)] = EMPTY_TOKEN
    day_1_dic[(user, 3)] = EMPTY_TOKEN
    day_1_dic[(user, 4)] = EMPTY_TOKEN
    day_1_dic[(user, 5)] = EMPTY_TOKEN
    day_1_dic[(user, 6)] = EMPTY_TOKEN
    day_1_dic[(user, 7)] = EMPTY_TOKEN
    day_1_dic[(user, 8)] = EMPTY_TOKEN
    day_1_dic[(user, 9)] = END_TOKEN
    


In [378]:
for data in day_1_lst :
    user = data[0]
    lst = data[1]
    
    for data2 in lst :
        if idx == 0 :
            first_week = data2[0]
            
            for idx2 in range(first_week) :
                day_1_dic[(user, idx2)] = START_TOKEN
                
        week = data2[0]
        day_1_dic[(user, week)] = data2[1:]

In [379]:
day_1_total_lst = []
temp1 = list(day_1_dic.values())

last1=0

In [380]:
for now in range(0,len(temp1)+1,9) :
    if now == 0 :
        last1 = now
        continue
    
    day_1_total_lst.append(temp1[last1:now])
    last1=now

In [381]:
print(len(day_1_total_lst))

13502
4880
2073
2108
1514
2294
3275
10354

40000


In [382]:
day_1_test_lst = np.array(day_1_total_lst)

<br></br><br></br>

# Predict

In [386]:
result_11 = np.argmax(day_1_model1.predict(day_1_test_lst), axis=1)
result_12 = np.argmax(day_1_model2.predict(day_1_test_lst), axis=1)
result_13 = np.argmax(day_1_model3.predict(day_1_test_lst), axis=1)

In [412]:
def voting(result_lst) :
    final_result = []
    
    for data in result_lst :
        temp = sorted([(0, data.count(0)),(1, data.count(1)),(2, data.count(2)),(3, data.count(3))], key=lambda x :x[1], reverse=True)
        final_result.append(temp[0][0])

    return pd.Series(final_result)

In [407]:
dic = {0 : "acc_id", 1:"model1", 2:"model2", 3:"model3", 4:"model4"}
result_1 = pd.DataFrame(list(zip(day_1_id_lst, result_11, result_12, result_13))).rename(columns = dic)

In [416]:
result_1["result"] = voting(list(zip(result_1.model1,result_1.model2,result_1.model3)))

In [443]:
result_lst = [result_1[["acc_id","result"]]]

total_result = pd.concat(result_lst).sort_values("acc_id")
total_result.head()

,acc_id,result
0,7,3
0,10,0
0,15,0
1,46,3
1,60,0


In [420]:
total_result.to_csv("./result/transformer_original_user_vector_with_decoder/transformer_original_user_vector_with_decoder.csv")

In [423]:
test_user_id_df = pd.read_csv("test_user_id.csv").drop("Unnamed: 0", axis=1)
test_user_id_df.head()

,acc_id,change_id
0,dadcb5d98e3142e64359841ebd2c0b28422ff86c37431e...,0
1,1e4daf084f5cf5a28dbaa678d1d07581e841fdcb8ba46c...,1
2,fe66a57a82633af556f7174bc4be15951b8f9ac5c2eaf2...,2
3,e967a7885ec55665406eed18a11d4e36de71c009697863...,3
4,73b98f746eb6764d5b628701b1aea48998cfac0ab7d923...,4


In [431]:
test_user_id_dic = {}

for kv in test_user_id_df.values :
    test_user_id_dic[kv[1]] = kv[0]

In [445]:
label_dic = {0 : "week", 1 : "month", 2:"2month", 3 :"retained"}

total_result["acc_id"] = total_result["acc_id"].map(lambda x: test_user_id_dic[x])
total_result["result"] = total_result["result"].map(lambda x: label_dic[x])
total_result.head()

,acc_id,result
0,0e777c479f0b38147d6b640cb4d529051e600d01c62515...,retain
0,da46b2a980f9d0fef41977e69437bce599e10f4ce423ae...,week
0,d3945da0721fc37d3c9adf94c591eefde1159767f8ae83...,week
1,8df710a3b6ac8e1065309bc6c3eabc9493b3842346dbe3...,retain
1,a0cff198321541502fd3940d072de55ce80a47d731bbd7...,week


In [446]:
total_result2 = total_result.set_index("acc_id").rename(columns = {"result" : "label"})
total_result2.head()

,label
acc_id,
0e777c479f0b38147d6b640cb4d529051e600d01c625155200f46b3becb2eb22,retain
da46b2a980f9d0fef41977e69437bce599e10f4ce423ae37312e35c80b9f4306,week
d3945da0721fc37d3c9adf94c591eefde1159767f8ae83e7aef9512ae17e421b,week
8df710a3b6ac8e1065309bc6c3eabc9493b3842346dbe36bccf17ee87a726891,retain
a0cff198321541502fd3940d072de55ce80a47d731bbd77bc598a71f0605cd09,week


In [447]:
total_result2.to_csv("./result/transformer_original_user_vector_with_decoder/transformer_original_user_vector_with_decoder_with_acc_id.csv")